In [11]:
import pandas as pd

df = pd.read_json('data\\post.json')
df.head(5)

,id,selftext,title,created_utc,upvote_ratio,num_comments,permalink
0,1ft9yze,Anyone else getting depressed when leaving the...,Server Leaving Blues,2024-09-30 23:32:21,0.82,11,/r/OnceHumanOfficial/comments/1ft9yze/server_l...
1,1ft5aab,I've stood outside another Rosetta's sealed up...,Is there a point in helping people guard devia...,2024-09-30 20:11:58,1.00,10,/r/OnceHumanOfficial/comments/1ft5aab/is_there...
2,1ft0ksl,Is it me or the server is very laggy sometimes...,Server Lag?,2024-09-30 16:59:48,0.80,17,/r/OnceHumanOfficial/comments/1ft0ksl/server_lag/
3,1fsrsfw,on theory it replace V and let u glide horizon...,prism clash: DO NOT get exploring drone,2024-09-30 09:53:24,0.75,16,/r/OnceHumanOfficial/comments/1fsrsfw/prism_cl...
4,1ftdsjr,,"My ""rare pattern"" collection so far...got abou...",2024-10-01 02:41:58,0.88,5,/r/OnceHumanOfficial/comments/1ftdsjr/my_rare_...


In [15]:
from sqlalchemy import create_engine

USERNAME = 'reddit'
PASSWORD = 'reddit'
DB = 'reddit'
HOST = 'localhost'
PORT = '5432'

engine = create_engine(f'postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}')

In [16]:
TABLE_NAME = 'reddit_posts'


df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False)

16

### Add Previous Data Before the Pipeline is fully built

In [21]:
import pandas as pd 

posts = pd.read_json('data\\posts.json')
comments = pd.read_json('data\\comments.json')

In [22]:
from sqlalchemy import create_engine

USERNAME = 'reddit'
PASSWORD = 'reddit'
DB = 'reddit'
HOST = 'localhost'
PORT = '5432'

engine = create_engine(f'postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}')

In [23]:
posts.to_sql(name='reddit_posts', con=engine, if_exists='append', index=False)

16

In [24]:
comments.to_sql(name='reddit_comments', con=engine, if_exists='append', index=False)

119

In [34]:
# past month data
comments_build = pd.read_json('data\\comments_build_20241002.json')
comments_discussion = pd.read_json('data\\comments_discussion_20241002.json')
posts_build = pd.read_json('data\\posts_build_20241002.json')
posts_discussion = pd.read_json('data\\posts_discussion_20241002.json')

In [26]:
posts_build.to_sql(name='reddit_posts', con=engine, if_exists='append', index=False)

28

In [43]:
# ensure that there's no intersection between discussion and build topics (because there's violation of primary key when loading data)
discussion_ids = set(posts_discussion['id'].values)
build_ids = set(posts_build['id'].values)
discussion_ids.intersection(build_ids)

set()

In [48]:
import numpy as np 

# some posts are already in the database so need to remove
excluded_ids = pd.read_sql_query('SELECT id FROM reddit_posts;', con=engine)
excluded_ids = excluded_ids.values.flatten()
excluded_ids

array(['1ftzssg', '1fu2yhu', '1ftm79i', '1ftx33d', '1ftyblc', '1fu62d2',
       '1fu9xt6', '1fu0d17', '1fu2hir', '1ftllod', '1fu5moa', '1fu564x',
       '1fu67an', '1ftzs26', '1ftxp3y', '1ftuqea', '1fu0ldl', '1ftp8n3',
       '1ftluuy', '1ft9yze', '1ft5aab', '1ft0ksl', '1fsrsfw', '1ftdsjr',
       '1ftdcuf', '1ft5d16', '1fsst6j', '1ft3q8e', '1ft48p3', '1ft8pm4',
       '1ft0yq9', '1fteerv', '1ft2hmv', '1ft5wri', '1fteype', '1f8fefx',
       '1fnxgds', '1fccv7q', '1fdf3xa', '1fa7ecn', '1fi1nep', '1f8xuxi',
       '1fakl3r', '1f7vman', '1fcih3c', '1fcclhh', '1fdaxwu', '1fe61jd',
       '1fs63sz', '1fln7ha', '1f712gz', '1f72had', '1fazf2m', '1fo5i4l',
       '1fitizx', '1fbj4p8', '1fe2q59', '1f80yxp', '1fq4hnv', '1fs4ju0',
       '1frkmjx', '1fpn3k5', '1fkf62m'], dtype=object)

In [50]:
posts_discussion = posts_discussion[~posts_discussion['id'].isin(excluded_ids)]

In [51]:
posts_discussion.to_sql(name='reddit_posts', con=engine, if_exists='append', index=False)

96

In [52]:
# same process for comments 
comments_build.to_sql(name='reddit_comments', con=engine, if_exists='append', index=False)

410

In [53]:
comments_discussion = comments_discussion[~comments_discussion['post_id'].isin(excluded_ids)]
comments_discussion.to_sql(name='reddit_comments', con=engine, if_exists='append', index=False)

459